In [2]:
import pandas as pd

# Reading the CSV files
whitney_artworks = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/whitney/artworks.csv')
whitney_artists = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/whitney/artists.csv')

# Convert 'id' column to 'artist_ids' as strings without .0 for integer values
whitney_artists['artist_ids'] = whitney_artists['id'].apply(lambda x: str(int(x)) if x.is_integer() else str(x))

# Ensure that 'artist_ids' column in whitney_artworks is of string type
whitney_artworks['artist_ids'] = whitney_artworks['artist_ids'].astype(str)

# Merging the two dataframes on 'artist_ids'
whitney = whitney_artworks.merge(whitney_artists, on="artist_ids", how='left')

# Displaying the result
whitney.head()
whitney_artists.to_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/whitney/whitney_data_raw.csv')

In [1]:
# tate dataset
import pandas as pd
tate_artworks = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/Tate/artwork_data.csv')
tate_artists = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/Tate/artist_data.csv')

# Convert 'id' column to 'artist_ids' as strings without .0 for integer values
tate_artists['ID'] = tate_artists['id']

# Ensure that 'artist_ids' column in tate_artworks is of string type
tate_artworks['ID'] = tate_artworks['artistId']

# Merging the two dataframes on 'artist_ids'
tate = tate_artworks.merge(tate_artists, on="ID", how='left')
tate.head()
tate.to_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/Tate/tate_raw.csv')


/var/folders/vh/0cdmtg494k7fs4m0gdhw8vtw0000gp/T/ipykernel_71060/3305933127.py:3: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  tate_artworks = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/Tate/artwork_data.csv')


In [3]:
#national gallery dataset

nga_artworks = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/national gallery (DC)/objects.csv')
nga_obj_const = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/national gallery (DC)/objects_constituents.csv')
nga_artists = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/national gallery (DC)/constituents.csv')

nga1 = nga_artworks.merge(nga_obj_const, on="objectid", how='left')

nga2 = nga1.merge(nga_artists, on="constituentid", how='left')
nga3 = nga2[nga2['roletype'] == 'artist']
nga_final = nga3.drop_duplicates(subset='objectid', keep='first')
nga_final.to_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/national gallery (DC)/national_gallery_raw.csv')

/var/folders/vh/0cdmtg494k7fs4m0gdhw8vtw0000gp/T/ipykernel_71060/1681967053.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  nga_artworks = pd.read_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/national gallery (DC)/objects.csv')


In [34]:
#flatten jsons and save them as csvs
import pandas as pd
from flatten_json import flatten
import json

data_paths_flattening = [
        "/Users/CUDAN/Documents/TLU/Data analysis/data/Kiasma/APIexample-master/dataset.json",  # Kiasma
        "/Users/CUDAN/Documents/TLU/Data analysis/data/smk/smk_all_da.json",
        "/Users/CUDAN/Documents/TLU/Data analysis/data/Kiasma/APIexample-master/dataset.json",  # Ateneum
    ]
flat_data = []
dictionary_data = []
for df in data_paths_flattening:
        with open(df, "r") as file:
            data = json.load(file)
            dictionary_data.append(data)

for df in dictionary_data:
        flattened_data = [flatten(record) for record in df]
        data = pd.DataFrame(flattened_data)
        flat_data.append(data)
        print(f"Loaded flattened data with shape: {data.shape}")

    # Separate kiasma and ateneum
flat_data[0] = flat_data[0][
        flat_data[0]["responsibleOrganisation"]
        == "Kansallisgalleria / Nykytaiteen museo Kiasma"
    ]
flat_data[2] = flat_data[2][
        flat_data[2]["responsibleOrganisation"]
        == "Kansallisgalleria / Ateneumin taidemuseo"
    ]

Loaded flattened data with shape: (58264, 499)
Loaded flattened data with shape: (175525, 2877)
Loaded flattened data with shape: (58264, 499)


In [16]:
test = flat_data[0].head(1000)

In [35]:
# simplify kiasma
# Extract the 'materials_' columns, row by row
# Join the values in each row into a single string
flat_data[0]['medium'] = flat_data[0].filter(regex='^materials_').apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
flat_data[0]['acquisition?'] = flat_data[0]['acquisitionYear']
# Drop columns matching regex patterns: 'keywords', 'multimedia', '_sv'
patterns_to_drop = ['keywords', 'multimedia', '_sv', 'materials', 'birthPlace', 'deathPlace', 'collection', 'role', 'attribution', 'acquisitionYear']
for pattern in patterns_to_drop:
    flat_data[0].drop(flat_data[0].filter(regex=pattern).columns, axis=1, inplace=True)
flat_data[0].dropna(axis=1, how='all', inplace=True)
flat_data[0]["artist_name"] = (
        flat_data[0]["people_0_firstName"] + " " + flat_data[0]["people_0_familyName"])

In [ ]:
# simplify ateneum
# Extract the 'materials_' columns, row by row
# Join the values in each row into a single string
flat_data[2]['medium'] = flat_data[2].filter(regex='^materials_').apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
# Drop columns matching regex patterns: 'keywords', 'multimedia', '_sv'
patterns_to_drop = ['keywords', 'multimedia', '_sv', 'materials', 'birthPlace', 'deathPlace', 'collection', 'role', 'attribution']
for pattern in patterns_to_drop:
    flat_data[2].drop(flat_data[2].filter(regex=pattern).columns, axis=1, inplace=True)
flat_data[2].dropna(axis=1, how='all', inplace=True)
flat_data[2]["artist_name"] = (
        flat_data[2]["people_0_firstName"] + " " + flat_data[2]["people_0_familyName"])


In [43]:
#simplify smk
#drop columns
flat_data[1].dropna(axis=1, how='all', inplace=True)
    
flat_data[1] = flat_data[1][['id', 'acquisition_date', 'dimensions_0_unit', 'dimensions_0_value', 'dimensions_1_unit', 
                            'dimensions_1_value', 'artist_0', 'titles_0_title', 'production_0_creator_gender',
                            'production_0_creator_nationality', 'credit_line_0', 'production_date_0_end', 
                            'production_0_creator_date_of_birth', 'production_0_creator_date_of_death',
                            'object_names_0_name', 'object_names_1_name']]



In [44]:
test = flat_data[1].head(1000)

In [45]:
flat_data[0].to_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/Kiasma/APIexample-master/kiasma_flattened.csv')
flat_data[1].to_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/smk/smk_flattened.csv')
flat_data[2].to_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/Kiasma/APIexample-master/ateneum_flattened.csv')

In [46]:
# simplify queensland
import re
# File path to the raw JSON data
data_path = "/Users/CUDAN/Documents/TLU/Data analysis/data/queensland/raw_data.json"

    # Load the JSON data
with open(data_path, "r") as file:
        data = json.load(file)

    # Extract the column names from the "fields" part of the JSON
columns = [field["id"] for field in data["fields"]]

    # Extract the records (the data)
records = data["records"]

    # Create the DataFrame
queensland = pd.DataFrame(records, columns=columns)
print(f"Loaded queensland with shape: {queensland.shape}")

    # Prepare queensland for analysis
def assign_dates(row, column_name):
        text = row[column_name]

        # Use regular expressions to find patterns for years
        years = re.findall(r"\b\d{4}\b", text)  # Find all 4-digit numbers

        # If exactly one year is found, it is the birth year
        if len(years) == 1:
            row["birth_date"] = int(years[0])
            row["death_date"] = None  # No death date if only one year is found
        # If exactly two years are found, assign the first to birth_date and the second to death_date
        elif len(years) >= 2:
            row["birth_date"] = int(years[0])
            row["death_date"] = int(years[1])
        else:
            # If no years or more than two years are found, set both to None
            row["birth_date"] = None
            row["death_date"] = None

        return row
queensland["Artist"] = queensland["Person"].str.split("\n").str.get(0)
queensland["Nationality"] = (
        queensland["Person"].str.split("\n").str.get(-1).str.split("1").str.get(0)
    )
queensland["birth_date"] = None
queensland["death_date"] = None
queensland = queensland.apply(assign_dates, axis=1, column_name="Person")
queensland.to_csv('/Users/CUDAN/Documents/TLU/Data analysis/data/queensland/queensland_flattened.csv')

Loaded queensland with shape: (20250, 17)
